In [ ]:
# ⬇️ Завантаження бібліотек
import pandas as pd
import re
import string

In [ ]:
# 📥 Завантаження файлу (якщо вручну з Colab)
from google.colab import files
uploaded = files.upload()

In [ ]:
# 📄 Читання CSV (файл має бути 'train.csv')
df = pd.read_csv('train.csv')

In [ ]:
# 🧼 Базова очистка — видаляємо пропущені значення
df = df.dropna(subset=["question1", "question2"])

In [ ]:
# 🔤 Функція препроцесінгу
def basic_preprocess(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# 🧽 Застосування до обох питань
df["question1_clean"] = df["question1"].apply(basic_preprocess)
df["question2_clean"] = df["question2"].apply(basic_preprocess)

In [ ]:
# 👀 Перевірка результатів
df[["question1", "question1_clean", "question2", "question2_clean"]].head()